In [1]:
from bs4 import BeautifulSoup
import requests as rq
import re
import datetime
import pymysql
from flask import Flask,jsonify, request
from fake_useragent import UserAgent
import cv2
import numpy as np
import time
from urllib.parse import urlencode
import threading
import sys
import os
import json
import shutil
import traceback
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import base64
os.environ['REQUESTS_CA_BUNDLE'] =  os.path.join(os.path.dirname(sys.argv[0]), 'cacert.pem')
print(os.path.join(os.path.dirname(sys.argv[0]), 'cacert.pem'))
import telnetlib
import configparser

D:\Program Files\anaconda3\lib\site-packages\cacert.pem


D:\Program Files\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Program Files\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\Program Files\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
try:
    # 建立 ConfigParser
    config = configparser.ConfigParser()
    read_file = input('請輸入設定檔檔名(gov_pur_settings.ini)：')
    if read_file=='':
        read_file='gov_pur_settings.ini'
    config.read(read_file)
    config_db_settings = config['db_settings']    
    db_settings = {
        "host": config_db_settings['host'],
        "port": config_db_settings.getint('port'),
        "user": config_db_settings['user'],
        "password": config_db_settings['password'],
        "db": config_db_settings['db'],
        "charset": config_db_settings['charset'],
        "autocommit":config_db_settings.getboolean('autocommit')
    }
    
    config_ip_time_use_interval = config['other_Settings'].getfloat('ip_time_use_interval')
    # 限制取得內頁資料的時間 (若成功抓取則等待)
    config_diff_seconds_data = config['other_Settings'].getfloat('diff_seconds_data')
    # 限制每一次嘗試取得內頁資料的時間 (每次嘗試連接都等待)
    config_diff_seconds_conn = config['other_Settings'].getfloat('diff_seconds_conn')
    # 設定是否用本機ip爬取內頁資料 (TRUE = 用本機ip)
    config_No_proxy = config['other_Settings'].getboolean('No_proxy')
    # 爬取幾筆就存進資料庫？
    config_block_crawl_no = config['other_Settings'].getint('block_crawl_no')
    # 請輸入主機ip ( 若要從本機啟動，可輸入127.0.0.1 )
    config_my_host = config['other_Settings']['my_host']
    # ip 允許 "不成功連結" 的時間
    config_ip_time_notConnect = config['other_Settings'].getint('ip_time_notConnect')
    # 創建一個資料夾存放驗證用撲克牌圖片
    config_path = config['other_Settings']['path']
    # 設定 ip qual_ratio
    config_ip_qual_ratio = config['other_Settings']['ip_qual_ratio']
    print(f"參數成功接收，嘗試連接 {db_settings['host']}:{db_settings['port']} {db_settings['db']} 資料庫")
except Exception as ex:
    print(ex)
    print('參數填寫錯誤')
        
try:
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
      #資料表相關操作
        # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
        command = f"SELECT VERSION()"
        # 執行
        cursor.execute(command)
        DB_VERSION = str(cursor.fetchone()[0])
        print(F"資料庫連線測試成功！(DB_VERSION：{DB_VERSION})")
except Exception as ex:
    print(f"資料庫連線測試失敗！，請檢查參數......{ex}")

請輸入設定檔檔名(gov_pur_settings.ini)：gov_pur_settings_2.ini
參數成功接收，嘗試連接 127.0.0.1:3306 tender_test 資料庫
資料庫連線測試成功！(DB_VERSION：8.0.27)


In [3]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "web.pcc.gov.tw",  #目標網站 
    "Connection":"close",
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
    "User-Agent":UserAgent(use_cache_server=False).random
     #使用者代理
}

In [4]:
#lxml pyinstaller編不動，nuitka不行
#parser = "lxml"
parser = "html.parser"
parser_detail = 'html5lib'

In [5]:
#創建一個資料夾存放驗證用撲克牌圖片
if os.path.exists(config_path):
    shutil.rmtree (config_path)
    os.mkdir(config_path)
else:
    os.mkdir(config_path)

In [6]:
# url = 'https://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F&primaryKey=53759591'
# urll='https://icanhazip.com/'
# s = rq.session()
# r = s.get(urll, headers = headers,verify=False,proxies={'https':'http://170.155.5.235:8080'}, timeout=(4,7))

In [7]:
def delete_ip(ip_time_notConnect,db_settings):
    print(f"ip_time_notConnect：{ip_time_notConnect}")
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = f"DELETE FROM {db_settings['db']}.proxy_ip WHERE TIMESTAMPDIFF(HOUR, updated_time, now()) >= {ip_time_notConnect};"
            # 執行
            try:
                cursor.execute(command)
            except Exception as err:
                print(err)
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "delete_wrong", "", str(err)))
    except Exception as ex:
        print(ex)

In [8]:
def test_ip(ip,port):
    try:
        telnetlib.Telnet(ip, port, timeout=1)
        return True
    except:
        return False

In [9]:
# 從免費代理網站取得代理ip
def get_proxy_ip(isHttps,db_settings):
    Now = datetime.datetime.now()
    url = {"usproxy":"https://www.us-proxy.org/","freeproxylist_sslproxy":"https://www.sslproxies.org/","freeproxylist_socksproxy":"https://www.socks-proxy.net/"}
    metas=[]
    for i in url:
        print(i)
        r = rq.get(url[i],verify=False, timeout=(4,15))
        html_doc = r.text
        soup = BeautifulSoup(html_doc, parser)
        trs = soup.select("table.table.table-striped.table-bordered tr")  
        #print(trs)
        for tr in trs:
            tds = tr.select("td")
            #print(tds)
            #print(len(tds))
            if len(tds) > 6:
                ifScheme = tds[6].text.lower()
                ip = tds[0].text
                port = tds[1].text
                # socks 代理會用到 version，sslproxy會抓錯，會抓成 anonymity
                version = tds[4].text.lower()
                proxy = "%s:%s"%(ip, port)
                meta = {
                    'proxyIp': proxy,
                    'connect_times':1,
                    'successful_connect_times':0,
                    'qual_ratio':0,
                    'isDelete':False,
                    'fromm':i,
                    'mine' : None,
                    'isValidate':test_ip(ip, port),
                    'updated_time':Now
                  }
                if ifScheme == isHttps:
                    if "sock" in version:
                        meta['ishttps'] = True
                        meta['proxyIp'] = version + "://" + meta['proxyIp']
                    elif ifScheme == "yes":
                        meta['ishttps'] = True
                        # https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#https-proxy-error-http-proxy
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    elif ifScheme == "no":
                        meta['ishttps'] = False
                        meta['proxyIp'] = "http://" + meta['proxyIp']
                    
                    #meta = test_proxy(meta,timeout_sec=3)
                    metas.append(meta)
                    continue                    
    #------------------------------------
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6", 
    "Host": "free-proxy.cz",  #目標網站 
    "Sec-Fetch-Dest": "document", 
    "Sec-Fetch-Mode": "navigate", 
    "Sec-Fetch-Site": "same-origin", 
    "Upgrade-Insecure-Requests": "?1", 
    "User-Agent":UserAgent(use_cache_server=False).random
     #使用者代理
}
    
    if isHttps =="yes":
        pp = 'https'
    elif isHttps =="no":
        pp = 'http'
    with rq.session() as s:
        s.keep_alive = True
        for j in range(1,4):
            try:
                r = s.get(f"http://free-proxy.cz/zh/",headers=headers,timeout=5)

                headers['Content-Type']='application/x-www-form-urlencoded'
                data=f"country=all&protocol={pp}&anonymity=all&send=Filter+proxies"
                r = s.post("http://free-proxy.cz/en/?do=searchFilter-submit",data=data,headers=headers,timeout=5)
                #http://free-proxy.cz/en/proxylist/country/all/https/ping/all/2
                if j >1:
                    headers['Referer']=f'http://free-proxy.cz/en/proxylist/country/all/{pp}/ping/all'
                    headers['Connection']=f'keep-alive'
                    r = s.get(f"http://free-proxy.cz/zh/proxylist/country/all/{pp}/ping/all/{j}",headers=headers,timeout=5,verify=False)
                print("free-proxy.cz抓取頁數： "+str(j))
            except Exception as e:
                print(e)
                print("free-proxy.cz 無法進入")
                break
            print(r.url)
            html_doc = r.text
            soup = BeautifulSoup(html_doc, parser)
            proxylist = soup.select('table#proxy_list tr')[1:]
            print(len(proxylist))
            for i in proxylist:
                try:
                    try:
                        ip = base64.b64decode(re.search(r'Base64\.decode\(\"(.*)\"\)',str(i.select('td')[0])).group(1)).decode("utf-8")
                    except AttributeError :
                        continue
                    port = i.select('td')[1].text
                    Protocol  = i.select('td')[2].text.lower()
                    place=i.select('td')[2].text.lower()
                    meta = {
                        'proxyIp': 'http://'+ str(ip) +':'+ str(port),
                        'connect_times':1,
                        'successful_connect_times':0,
                        'qual_ratio':0,
                        'isDelete':False,
                        'fromm':'freeProxyCZ',
                        'mine' : None,
                        'isValidate':test_ip(ip, port),
                        'updated_time':Now
                      }
                    if Protocol =='https' and isHttps =="yes":
                        meta['ishttps'] = True
                        # 測了就抓不到下一頁了
                        #meta = test_proxy(meta,timeout_sec=3)
                        metas.append(meta)
                    elif Protocol =='http' and isHttps =="no":
                        meta['ishttps'] = False
                        #meta = test_proxy(meta,timeout_sec=3)
                        metas.append(meta)
                except:
                    continue
                if len(proxylist)<37:
                    break
    return metas

In [10]:
# 測試 proxy 可用性
def test_proxy(proxy,timeout_sec,headers={}):
    #print(proxy['proxyIp'])
    with rq.session() as s:
        try:
            if proxy['ishttps'] == True:
                url = "https://icanhazip.com/"
                r = s.get(url, headers = headers,proxies={'https':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
            elif proxy['ishttps'] == False:
                url = "http://icanhazip.com/"
                r = s.get(url, headers = headers,proxies={'http':proxy['proxyIp'],},verify=False, timeout=timeout_sec)
            print(r.status_code)
            print(proxy['proxyIp'])
            if r.status_code==200:
                proxy['isValidate']=True
                proxy['successful_connect_times']=1
                proxy['qual_ratio']=1
            else:
                proxy['isValidate']=False
        except Exception as ex:
            proxy['isValidate']=False
            #print(ex)
        #print("---------------")
    return proxy

In [11]:
# 將取得的代理ip寫入資料庫，此函數也可以更新已寫入資料庫的資料
def send_ip(proxies_pool_https, db_settings,typee="",send_log = True,new = False):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    wrong = 0
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id=f"select MAX(UID) FROM {db_settings['db']}.log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = f"INSERT INTO {db_settings['db']}.proxy_ip(log_UID, proxyIp, connect_times, successful_connect_times, qual_ratio, isValidate, ishttps,isDelete, fromm, updated_time)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),connect_times =connect_times+values(connect_times),successful_connect_times=successful_connect_times+values(successful_connect_times),qual_ratio=((successful_connect_times+values(successful_connect_times))/(connect_times+values(connect_times))),isValidate = values(isValidate),ishttps=values(ishttps),isDelete=values(isDelete),updated_time=values(updated_time)"
            if new:
                command = f"INSERT INTO {db_settings['db']}.proxy_ip(log_UID, proxyIp, connect_times, successful_connect_times, qual_ratio, isValidate, ishttps,isDelete, fromm, updated_time)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),connect_times = values(connect_times),successful_connect_times=values(successful_connect_times),qual_ratio=values(qual_ratio),isValidate = values(isValidate),ishttps=values(ishttps),isDelete=values(isDelete),updated_time=values(updated_time)"
            command_log = f"INSERT INTO {db_settings['db']}.log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始 (暫時取消)
#             if send_log:
#                 cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "start", typee, ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 組合數據
            if type(proxies_pool_https) == list:
                for proxy in proxies_pool_https:
                    data_combine.append((log_UID, proxy["proxyIp"], proxy["connect_times"], proxy["successful_connect_times"], proxy["qual_ratio"], proxy["isValidate"], proxy["ishttps"], proxy["isDelete"], proxy["fromm"], proxy["updated_time"]))  # 注意要用两个括号扩起来
            else:
                data_combine.append((log_UID, proxies_pool_https["proxyIp"], proxies_pool_https["connect_times"], proxies_pool_https["successful_connect_times"], proxies_pool_https["qual_ratio"], proxies_pool_https["isValidate"], proxies_pool_https["ishttps"], proxies_pool_https["isDelete"], proxies_pool_https["fromm"]))  # 注意要用两个括号扩起来
            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as err:
                print(err)
                wrong = wrong + 1
                print(wrong)
                cursor.execute(command_log, (datetime.datetime.now(), "proxy_ip", "wrong" + str(wrong), "", str(err)))
    except Exception as ex:
        print(ex)
        print(data_combine)
    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('proxy ip寫入資料庫，time cost', time_c, 's')

In [12]:
# p_yes=get_proxy_ip("yes",db_settings)
# send_ip(p_yes, db_settings ,"only https",new = True)
# x=get_ip_from_db("yes",db_settings,config_ip_qual_ratio,config_ip_time_use_interval,connect_times=1,new=True)

In [13]:
# 從資料庫取出ip，未來有自架proxy的話，可以把proxy放到資料庫內，並設定 mine = 1，這樣就會固定抓這個proxy來用，如果沒有自架proxy的話，就還需要多寫 proxy expire 的處置方式。
def get_ip_from_db(isHttps,db_settings,ip_qual_ratio,ip_time_use_interval,connect_times=1,new=True):
    print(f'ip_qual_ratio：{ip_qual_ratio}')
    print(f'ip_time_use_interval：{ip_time_use_interval}')
    try:
        # 建立Connection物件
        data=[]
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
          #資料表相關操作
            if isHttps=="yes":
                get_ip=f"select * FROM {db_settings['db']}.proxy_ip where isHttps = 1 and isDelete is False and ((qual_ratio > {ip_qual_ratio} or connect_times < {connect_times}) or (mine = 1 and TIMESTAMPDIFF(second, updated_time, now())>{ip_time_use_interval}))"
            elif isHttps=="no":
                get_ip=f"select * FROM {db_settings['db']}.proxy_ip where isHttps = 0 and isDelete is False and ((qual_ratio > {ip_qual_ratio} or connect_times < {connect_times}) or (mine = 1 and TIMESTAMPDIFF(second, updated_time, now())>{ip_time_use_interval}))"
            cursor.execute(get_ip)
            datas = cursor.fetchall()
        if new:
            for data in datas:
                data['successful_connect_times']=0
                data['connect_times']=0
                data['qual_ratio']=0
        return datas
    except Exception as ex:
        print(f'ip抓取失敗：{ex}')

In [14]:
#更新 mine proxy_ip 的可取用時間
def update_mine_proxy_ip(proxy_ip,db_settings):
    conn = pymysql.connect(**db_settings)
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:
        update_proxy_ip = f"update {db_settings['db']}.proxy_ip SET updated_time=now() where proxyIp = '{proxy_ip['proxyIp']}'"
        cursor.execute(update_proxy_ip)

In [15]:
#撲克牌圖片驗證用算法
def pHash(img):
    # 感知哈希算法
    # 缩放32*32
    img = cv2.resize(img, (32, 32))   # , interpolation=cv2.INTER_CUBIC
 
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:8, 0:8]
 
    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [16]:
#撲克牌圖片驗證用算法 - 比較
def cmpHash(hash1, hash2):
    # Hash值对比
    # 算法中1和0顺序组合起来的即是图片的指纹hash。顺序不固定，但是比较的时候必须是相同的顺序。
    # 对比两幅图的指纹，计算汉明距离，即两个64位的hash值有多少是不一样的，不同的位数越小，图片越相似
    # 汉明距离：一组二进制数据变成另一组数据所需要的步骤，可以衡量两图的差异，汉明距离越小，则相似度越高。汉明距离为0，即两张图片完全一样
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1) != len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 不相等则n计数+1，n最终为相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n

In [17]:
#進行撲克牌驗證
def validate_ip(s,url,proxies,path):
    response=""
    x=0
    while x <= 5:
        response = s.get(url, proxies=proxies)
        html_doc = response.text
        soup = BeautifulSoup(html_doc, parser)
        img_url_d = soup.select('form#validateForm img')
        img_url_head = "http://web.pcc.gov.tw/tps/tpam/"
        img_data = []
        img_data.append({'img_src':img_url_head +img_url_d[0]['src'],'id':'question'})
        for j in img_url_d[2:]:
            d = {'img_src':img_url_head +j['src'],'id':j['alt']}
            img_data.append(d)
        right_hash_com =[]
        left_hash_com =[]
        
        for k in img_data:
            response = s.get(k['img_src'])
            with open(path +'/'+ k['id']+'.jpg', 'wb') as file:
                file.write(response.content)
                file.close() 
            if k ==img_data[0]:
                img = cv2.imread(path +'/'+"question.jpg")
                crop_img = img[0:96, 6:77]
                ret,crop_img = cv2.threshold(crop_img,127,255,cv2.THRESH_BINARY)
                right_imageVar = pHash(crop_img)
                crop_img = img[0:96, 89:160]
                ret,crop_img = cv2.threshold(crop_img,127,255,cv2.THRESH_BINARY)
                left_imageVar = pHash(crop_img)
            image = cv2.imread(path +'/'+k['id']+'.jpg')
            ret,image = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
            right = cmpHash(pHash(image),right_imageVar)
            left = cmpHash(pHash(image),left_imageVar)
            right_hash_com.append(right)
            left_hash_com.append(left)

        right_idx = np.argmin(right_hash_com[1:])
        left_idx = np.argmin(left_hash_com[1:])
        data="choose=" +img_data[right_idx+1]['id']+"&choose="+img_data[left_idx+1]['id']+"&id="+re.match(r'.*id=(.*)', url, flags=0).group(1)
        response = s.post(url,headers = {'Content-Type':'application/x-www-form-urlencoded'}, proxies=proxies,data=data, timeout=(5,150))
        if re.search("為預防惡意程式針對本系統進行大量查詢致影響系統服務品質",response.text):
            print(f"再次檢核：{response.url}")
            url=response.url
        elif not re.search("為預防惡意程式針對本系統進行大量查詢致影響系統服務品質",response.text) and response.status_code == 200:
            print("通過驗證")
            #print(response.text)
            break
        else:
            #print(response.text)
            x=x+1
            print(f"驗證失敗，{response.status_code}")

    return response

In [18]:
#取出字典某 key 值對應 value，若找無此 key 值，則回傳空字串 
def dicMemberCheck(key, dicObj):
    if key in dicObj:
        return dicObj[key]
    else:
        return ""
# dicMemberCheck('標案案號', {'標案案號':123,'ji3':456})

In [19]:
# href = "http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmHistoryAction.do?method=review&searchMode=common&pkPmsMainHist=null&pkPmsMain=53735656&awardNoticeDateOrgn=111/03/16"
# r=rq.get(href)

In [20]:
# href = "https://t2.pcc.gov.tw/tps/QueryTender/query/searchTenderDetail?pkPmsMain=NjAwMTU1Nzk="

# s = rq.session()
# proxies = get_ip_from_db("yes",db_settings,0.3,800)
# refresh_proxies=[]
# ran=None
# (detail_data,detail_data_html,proxies,refresh_proxies,ran) = get_tenderDeclaration_detail(s,href,None,proxies,refresh_proxies,db_settings,diff_seconds_conn=0,headers={},typee="tenderDeclaration",No_proxy=True)

# conn = pymysql.connect(**db_settings)
# get_data="SELECT * FROM gov_purchase where (detail_connect like '%=NjAwMTU1Nzk=') "
# # 建立Cursor物件
# with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#   #資料表相關操作
#     cursor.execute(get_data)
#     raw_datas_all_3 = cursor.fetchall()
    
# str_detail_data_html=str(detail_data_html)
# if len(detail_data)>0 and (re.search(raw_data['bid_no'],str_detail_data_html) or re.search(raw_data['proposer_name'],str_detail_data_html)):
#     print("1")

In [21]:
# tenderType = 'tenderDeclaration'
# conn = pymysql.connect(**db_settings)
# get_data="SELECT * FROM gov_purchase where (detail_data_html ='' and detail_data like '%{}%') and tenderType='"+tenderType+"'"
# # 建立Cursor物件
# with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#   #資料表相關操作
#     cursor.execute(get_data)
#     raw_datas_all = cursor.fetchall()
# raw_datas = raw_datas_all[0:10]
# # get_only_detail(tenderType,raw_datas_all,0,0,headers, db_settings,No_proxy=True)

In [22]:
# s=rq.session()
# proxies=get_ip_from_db("yes",db_settings,0.3)
# refresh_proxies=[]
# good=[]
# ran=None
# url = 'https://t2.pcc.gov.tw/prkms/urlSelector/common/atm?pk=NjAwMDUwNjA='
# ll = get_only_detail(tenderType,raw_datas,0,0,headers, db_settings,No_proxy=True)
# # get_only_detail會回傳成功抓取的筆數 (ll)

In [23]:
# xx=json.loads(raw_datas_all[0]['detail_data'])
# raw_datas_all[0]['detail_data']=json.dumps(xx,ensure_ascii=False)

In [24]:
# href="https://web.pcc.gov.tw/tps/tpam/main/tps/tpam/tpam_tender_detail.do?searchMode=common&scope=F&primaryKey=53789621"
# href2="https://www.youtube.com/watch?v=uV2rUMcGBYI"
# proxies=get_ip_from_db("yes",db_settings,0.3)
# proxies=proxies[:30]
# good=[]
# for i in proxies:
#         try:
#         r = rq.get(href, headers = headers,verify=False,proxies={'https':i['proxyIp']}, timeout=(6,15))
#         print(str(i['proxyIp'])+" !!!!!!!!!!!!!")
#         good.append(i)
#     except:
#         print(str(i['proxyIp'])+" .............")

In [25]:
# 抓招標、決標內頁資料
def get_tenderDeclaration_detail(s,href,ran,proxies,refresh_proxies,db_settings,headers,typee,path,diff_seconds_conn=1.2,No_proxy=False):
    detail_data_html=""
    detail_data={}
    global detail_waitt
    detail_waitt = 0
    #with requests.Session() as s:
    headers["User-Agent"] = UserAgent(use_cache_server=False).random
    breakdown = "no"
    time.sleep(diff_seconds_conn)
    if No_proxy:
        while detail_waitt:
            localtime = time.localtime()
            result = time.strftime("%Y%m%d_%H%M%S", localtime)
            print(typee + f"暫停休息 {60*detail_waitt}秒 一下，有執行緒被鎖，目前時間：{result}")
            time.sleep(60*detail_waitt)
        while True:
            try:
                r = s.get(href, headers = headers,verify=False, timeout=(4,15))
                html_doc = r.text
                soup = BeautifulSoup(html_doc, parser_detail)
                if re.match(r'.*validate.*',r.url, flags=0):
                    print(f"本機 IP 準備執行驗證")
                    r = validate_ip(s,r.url,{},path)
                if r.status_code<300 and (re.search(r'公告日',r.text) or re.search(r'標案案號',r.text)):
                    detail_waitt = 0
                    break
                elif re.search(r'系統錯誤訊息',r.text):
                    detail_waitt = 0
                    error_section = soup.select("table.g_tb_01 tr")
                    for i in error_section:
                        error_1 = i.select("td")[0].text.strip()
                        error_2 = i.select("td")[1].text.strip()
                        detail_data[error_1]=error_2
                    breakdown = "yes"
                    break
            except rq.exceptions.RequestException as ex:
                print("本機 ip 遭到封鎖")
                detail_waitt = detail_waitt + 1
                localtime = time.localtime()
                result = time.strftime("%Y%m%d_%H%M%S", localtime)
                print(typee + f"暫停休息 {800*detail_waitt}秒 一下，本機 IP 被鎖，目前時間：{result}")
                print(ex)
                time.sleep(800*detail_waitt)
            except Exception as ex:
                print(f'其他錯誤_1：{ex}')
                break
    else:
        while True:
            if not ran:
                ran = np.random.randint(len(proxies))
            print(f"抽取第 {ran} 個，目前共有 {len(proxies)} 個 proxy，當前連接次數：{proxies[ran]['connect_times']}")
            try:
                proxies[ran]['connect_times'] =  proxies[ran]['connect_times'] + 1
                proxies[ran]['qual_ratio'] = proxies[ran]['successful_connect_times'] / proxies[ran]['connect_times']
                #print(f"{proxies[ran]['proxyIp']} 準備進入，{href}")                 
                r = s.get(href, headers = headers,verify=False,proxies={'https':proxies[ran]['proxyIp']}, timeout=(8,15))
                html_doc = r.text
                soup = BeautifulSoup(html_doc, parser_detail)
                #print("內頁取得點存活")
                if re.match(r'.*validate.*',r.url, flags=0) or re.search("為預防惡意程式針對本系統進行大量查詢致影響系統服務品質",r.text):
                    print(f"{proxies[ran]['proxyIp']} 準備執行驗證")
                    r = validate_ip(s,r.url,proxies={'https':proxies[ran]['proxyIp']})
                if re.search(r'公告日',r.text):
                    proxies[ran]['successful_connect_times'] =  proxies[ran]['successful_connect_times'] + 1
                    proxies[ran]['qual_ratio'] = proxies[ran]['successful_connect_times'] / proxies[ran]['connect_times']
                    proxies[ran]['updated_time'] = datetime.datetime.now()
                    #避免鎖 ip 的情況，所以限定每個 ip 爬取 50 筆資料
                    if proxies[ran]['successful_connect_times']>=50:
                        refresh_proxies.append(proxies.pop(ran))
                        ran=None
                        print("此 ip 已爬取 50 個")
                        if len(proxies) == 0:
                            breakdown = "yes"
                            print("proxy 不夠用了，重新獲取")
                    break
                elif re.search(r'系統錯誤訊息',r.text):
                    error_section = soup.select("table.g_tb_01 tr")
                    for i in error_section:
                        error_1 = i.select("td")[0].text.strip()
                        error_2 = i.select("td")[1].text.strip()
                        detail_data[error_1]=error_2
                    breakdown = "yes"
                    break
                else:
                    refresh_proxies.append(proxies.pop(ran))
                    ran=None
                    # 被chrome擋住 or validate_ip 執行太多次
                    print(str(refresh_proxies[-1]['proxyIp']) + " 內頁資料錯誤，還剩餘 " + str(len(proxies)) +" 個proxy")
                    print(r.url)
                    print(r.text)
                    if len(proxies) == 0:
                        breakdown = "yes"
                        print("proxy 不夠用了，重新獲取")
                        break
            except rq.exceptions.RequestException as ex:
                if proxies[ran]['mine']==1:
                    update_mine_proxy_ip(proxies[ran],db_settings)
                #print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                #print(errMsg)
                refresh_proxies.append(proxies.pop(ran))
                print(str(refresh_proxies[-1]['proxyIp']) + " 連接失敗，還剩餘 " + str(len(proxies)) +" 個proxy")
                ran=None
                if len(proxies) == 0:
                    breakdown = "yes"
                    print("proxy 不夠用了，重新獲取")
                    break
            except Exception as ex:
                print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print("其他錯誤_2：" + errMsg)
                ran=None
    if breakdown == "yes":
        return detail_data,detail_data_html,proxies,refresh_proxies,None

    if typee=="predict":
        #政府採購預告
        detail_data_html = soup.select('div#printRange')[0]
        for i in detail_data_html.select('table'):
            for j in i.select('tr'):
                try:
                    col = j.select('th')[-1].text.strip()
                except:
                    col = j.select('td')[-2].text.strip()
                val = j.select('td')[-1].text.strip()
                detail_data[col]=val

    elif typee =="final":

        # 決標公告：無法決標、撤銷公告
        if soup.select('div#printArea.main div#hidden_message_id'):
            detail_data_html = soup.select('div#printArea.main table')[0]
            detail_data['really_final']=0
            block_data = detail_data_html.select('tr')
            if not block_data[0].text.strip():
                del block_data[0]
            if re.search(r'紅色字體表示此次更正公',block_data[0].text):
                del block_data[0]
            for j in block_data:
                try:
                    try:
                        col = j.select('th')[-1].text.strip()
                    except:
                        col = j.select('td')[-2].text.strip()
                    val = j.select('td')[-1].text.strip()
                    detail_data[col]=val
                except:
                    #https://web.pcc.gov.tw/tps/main/pms/tps/atm/atmNonAwardAction.do?searchMode=common&method=nonAwardContentForPublic&pkAtmMain=53705535
                    #https://web.pcc.gov.tw/tps/main/pms/tps/atm/atmNonAwardAction.do?searchMode=common&method=nonAwardContentForPublic&pkAtmMain=53705381
                    print("其他錯誤_3")
                    print(r.url)
                    print(j)
                    pass
                
        else:
            #決標公告：正常決標公告 (NewSite down)，未處理更正決標公告的部分
            detail_data_html = soup.select('div#printRange')[0]
            detail_data['really_final']=1
            for idx_1,i in enumerate(detail_data_html.select('div#friendlyPrintRange table.tableShow')):
                for idx_2,j in enumerate(i.select('tr')):
                    if idx_2 == 0:
                        class_name=j.select("td")[0].text
                        continue
            #                 if idx == 0 and re.search(r'紅色字體表示此次更正公',block_data[0].text):
            #                     del block_data[0]
            #                 if len(block_data) == 0 :
            #                     continue
            #                 else:
            #                     del block_data[0]
                    script_tag = j.find('script')
                    if script_tag:
                        script_tag.decompose()
                    try:
                        col = j.select('td')[0].text.strip()
                        val = j.select('td')[1].text.strip()
                        detail_data[col]=val
                    except:
                        print(f"決標公告 內頁資料 解析失敗，出現在 {r.url} ，第{idx_1}個 table，第{idx_2}個 tr")   
                        print(j)                    
                        
    else:
        #招標公告：電子競價公告 (NewSite down) 沒有公告日期...
        if re.search("電子競價公告",html_doc):
            detail_data_html = soup.select('div.middle_1_cen_big_full')[0]
            block_data = detail_data_html.select('table.tb_01')
            for idx, j in enumerate(block_data[0].select('tr')):
                if idx ==0:
                    class_name=j.select("td")[0].text
                    continue
                try:
                    col = j.select('td')[0].text.strip()
                    try:
                        val = j.select('td')[1].text.strip()
                    except:
                        continue
                    detail_data[col]=val
                except:
                    print(f"招標公告 內頁資料 解析失敗，出現在 {r.url}，第{i}個 table，第{j}個 tr")   
        else:
            #招標公告 (NewSite down)
            detail_data_html = soup.select('div#printRange')[0]
            #加上公告日期
            publish_date = re.match(r'([\s\S]*)var targetDate = dateTransMinGo\(\"(.*)\"\);',html_doc).group(2)
            publish_date = publish_date.replace(publish_date[:4],str(int(publish_date[:4])-1911),1)
            detail_data_html = str(detail_data_html).replace('<div id="printRange">',f'<div id="printRange"><div style="text-align: center; width: 100%">公告日：{publish_date}</div>',1)
            detail_data_html = BeautifulSoup(detail_data_html, parser_detail)
            for i in range(1,10):
                block_data = detail_data_html.select('table.tb_0'+str(i))
                if len(block_data) == 0 :
                    continue

                for idx, j in enumerate(block_data[0].select('tr')):
                    if idx ==0:
                        class_name=j.select("td")[0].text
                        continue
                    try:
                        col = j.select('td')[0].text.strip()
                        try:
                            val = j.select('td')[1].text.strip()
                        except:
                            continue
                        detail_data[col]=val
                    except:
                        print(f"招標公告 內頁資料 解析失敗，出現在 {r.url}，第{i}個 table，第{j}個 tr")         
                         
    return detail_data,detail_data_html,proxies,refresh_proxies,ran

In [26]:
def get_only_detail(tenderType,raw_datas,diff_seconds_data,diff_seconds_conn,headers, db_settings,path,No_proxy=False):
    global progress_final_detail
    global progress_tenderDeclaration_detail
    proxies = get_ip_from_db("yes",db_settings,config_ip_qual_ratio,config_ip_time_use_interval)
    refresh_proxies=[]
    longg = len(raw_datas)
    loses=[]
    x=0
    ran=None
    with rq.session() as s:
        s.mount('https://', requests.adapters.HTTPAdapter(pool_connections=25, pool_maxsize=50))
        s.mount('http://', requests.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=40))
        s.mount('https://', requests.adapters.HTTPAdapter(max_retries=1))
        s.mount('http://', requests.adapters.HTTPAdapter(max_retries=1))
        s.keep_alive = False
        for raw_data in raw_datas:
            time_start = datetime.datetime.now() #開始計時
            print("--------------------")
            try:
                while True:
                    if tenderType =='final' or tenderType =='tenderDeclaration':
                        (detail_data,detail_data_html,proxies,refresh_proxies,ran) = get_tenderDeclaration_detail(s, raw_data['detail_connect'],ran,proxies,refresh_proxies,db_settings,headers,raw_data['tenderType'],path,diff_seconds_conn,No_proxy=No_proxy) 
                    elif tenderType =='searchAppeal':
                        (detail_data,detail_data_html) = get_searchAppeal_detail(headers,raw_data['detail_connect'])
                    str_detail_data_html=str(detail_data_html)
                    if len(detail_data)>0 and (re.search(raw_data['bid_no'],str_detail_data_html) or re.search(raw_data['proposer_name'],str_detail_data_html)):
                        raw_data['detail_data'] = detail_data
                        raw_data['detail_data_html'] = detail_data_html
                        print(raw_data['detail_data']['機關代碼'])
                        
                        time_end = datetime.datetime.now()    #結束計時
                        diff=time_end-time_start
                        if diff.seconds<diff_seconds_data:
                            time.sleep(diff_seconds_data-diff.total_seconds())
                        time_end = datetime.datetime.now()    #結束計時
                        time_c= time_end - time_start   #執行所花時間
                        if tenderType=='tenderDeclaration':
                            progress_tenderDeclaration_detail = f'progress_tenderDeclaration_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_tenderDeclaration_detail)
                        elif tenderType=='final':
                            progress_final_detail = f'progress_final_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_final_detail)
                        loses=[]
                        x=x+1
                        break
                    elif "錯誤代碼" in str(detail_data.keys()) or "錯誤訊息" in str(detail_data.keys()):
                        raw_data['error_code'] = 1
                        time_end = datetime.datetime.now()#結束計時
                        diff=time_end-time_start
                        if diff.seconds<diff_seconds_data:
                            time.sleep(diff_seconds_data-diff.total_seconds())
                        time_end = datetime.datetime.now()    #結束計時
                        time_c= time_end - time_start   #執行所花時間
                        if tenderType=='tenderDeclaration':
                            progress_tenderDeclaration_detail = f'progress_tenderDeclaration_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，系統發生錯誤，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_tenderDeclaration_detail)
                        elif tenderType=='final':
                            progress_final_detail = f'progress_final_detail 目前是第 {x+1} 項，抓取該細項內頁花費 {time_c} 秒，系統發生錯誤：{raw_data}，進度：{(x+1)*100/longg}%，總共有 {longg} 項'
                            print(progress_final_detail)
                        loses=[]
                        x=x+1
                        break
                    elif len(proxies)==0:
                        send_ip(refresh_proxies, db_settings,send_log=False)
                        
                        # 看一下存 ip 的時候到底丟了甚麼鬼進去
                        localtime = time.localtime()
                        result = time.strftime("%Y%m%d_%H%M%S", localtime)
                        with open('ip_'+result+'.txt', 'a+', encoding='utf-8') as file:
                            file.write(str(refresh_proxies))
                            file.close()
                            
                        loses.append(1)
                        #print(f"失敗總次數：{len(loses)}，危險！")
                        proxies = get_ip_from_db("yes",db_settings,config_ip_qual_ratio,config_ip_time_use_interval)
                        print(f"成功獲取 proxy：{len(proxies)} 個")
                        # 避免既有的proxy在qual_ratio的情況下突然失效，要讓這種proxy的qual_ratio降到不會被抓取的水平要很久，所以設置如果 7次都用光proxy就重爬proxy一次
                        if len(proxies)==0 or len(loses)>7:
                            print(f"失敗總次數：{len(loses)}，強制重抓proxy！！！！！！！！！！")
                            loses=[]
                            proxies = get_proxy_ip("yes",db_settings)
                            send_ip(proxies, db_settings,new = True)
                    else:
                        raw_data['error_code'] = 2
                        loses=[]
                        localtime = time.localtime()
                        result = time.strftime("%Y%m%d_%H%M%S", localtime)
                        with open(str(raw_data['bid_no'])+'_'+result+'.txt', 'a+', encoding='utf-8') as file:
                            file.write(str(raw_data))
                            file.close()
            except Exception as ex:
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print(errMsg)
                print("其他錯誤_6："+str(raw_data))
                pass
    send_ip(refresh_proxies, db_settings,send_log=False)

    if x>=1:
        try:
            # 寫入要更新 detail 資料，所以 Not_update_detail=False
            send_db(raw_datas,"detail_data_"+tenderType, db_settings,Not_update_detail=False)
        except Exception as ex:
            print("其他錯誤_7：招標決標寫入DB 失敗")
            localtime = time.localtime()
            result = time.strftime("%Y%m%d_%H%M%S", localtime)
            with open('send_db_2_' + result+'.txt', 'a+', encoding='utf-8') as file:
                file.write(str(raw_datas))
                file.close() 
            with open('send_db_1_' + result+'.txt', 'a+', encoding='utf-8') as file:
                file.write(str(ex))
                file.close() 
    return x

In [27]:
# 抓公開徵求內頁資料
def get_searchAppeal_detail(headers,detail_connect):
    detail_data={}
    detail_data_html=""
    x=0
    while True:
        try:
            x=x+1
            r = rq.get(detail_connect, headers = headers)
            html_doc = r.text
            soup = BeautifulSoup(html_doc, parser)
            if r.status_code == 200 and re.search(r'公告日',html_doc):
                x=0
                break
            elif re.search(r'系統錯誤訊息',html_doc):
                detail_waitt = 0
                error_section = soup.select("table.g_tb_01 tr")
                for i in error_section:
                    error_1 = i.select("td")[0].text.strip()
                    error_2 = i.select("td")[1].text.strip()
                    detail_data[error_1]=error_2
                return detail_data,detail_data_html
            else:
                x=0
        except Exception as ex:
            time.sleep(1201*x)
            print(ex)
            print(f"get_searchAppeal_detail出錯，等待 {1201*x} 秒")


    if re.search(r'無符合此筆公開徵求廠商條件資料',html_doc):
        print("其他錯誤_5：無符合此筆公開徵求廠商條件資料，可能公告已撤銷")
        detail_data_html = soup.select('center table')[0]
        detail_data=""
        return detail_data,detail_data_html
    try:
        detail_data_html = soup.select('div#printRange')[0]
    except:
        print("----------------")
        print(soup)

    for i in detail_data_html.select('tr'):
        try:
            col = i.select('th')[-1].text.strip()
        except:
            col = i.select('td')[0].text.strip()
        val = i.select('td')[-1].text.strip()
        detail_data[col]=val
    
    return detail_data,detail_data_html

In [28]:
# 抓公開閱覽內頁資料
def get_publicRead_detail(headers, detail_connect):
    x=0
    while True:
        try:
            x=x+1
            r = rq.get(detail_connect, headers = headers)
            if r.status_code == 200:
                x=0
                break
        except Exception as ex:
            time.sleep(1201*x)
            print(ex)
            print(f"get_publicRead_detail出錯，等待 {1201*x} 秒")
    
    html_doc = r.text
    soup = BeautifulSoup(html_doc, parser)
    detail_data_html = soup.select('div div.middle_1_cen_big')[0]
    
    # 為了顯示公告日期
    for prev_sibling in detail_data_html.select("div#printRange")[0].find_previous_siblings()[1:]:
        prev_sibling.decompose()
        
    for next_sibling in detail_data_html.select("div#printRange")[0].find_next_siblings():
        next_sibling.decompose()
    
    #拿掉上邊框
    detail_data_html = str(detail_data_html).replace("margin-top: 30px;","",1)
    #拿掉表格邊距
    detail_data_html = str(detail_data_html).replace("margin-top:10px","width: 100%;",1)
    
    detail_data_html = BeautifulSoup(detail_data_html, parser)
    
    detail_data={}
    for i in detail_data_html.select('tr'):
        col = i.select('td.tbg_1')[0].text.strip()
        val = i.select('td.tbg_2')[0].text.strip()
        detail_data[col]=val
    return detail_data,detail_data_html

In [29]:
# current_date = (datetime.date.today()- datetime.timedelta(days=30)).strftime("%Y/%m/%d")
# current_date = str(int(current_date[0:4])-1911) + current_date[4:]
# print(current_date)
# s = rq.session()

# url_first = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"
# data={'method':'search','searchMethod':'true','searchTarget':'ATM','hid_1':1,'hid_2':1,'hid_3':1,'tenderStatus':'5,6,20,28,8,21,22,29,33,9,23','btnQuery':'查詢','awardAnnounceStartDate':current_date,'awardAnnounceEndDate':current_date}
# r = s.post(url_first,data = data)

# url = "https://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&execLocationArea=&pageIndex=1"
# r = s.get(url)
# soup = BeautifulSoup(r.text, parser)
# data = soup.select("div#print_area table tr")

# for i in data[1:3]:
#     #機關名稱
#     proposer_name=i.select('td')[1].text.strip()
#     print(proposer_name)
#     #標案案號
#     bid_no=re.match(r'(.*)\r\n\t', i.select('td')[2].text.strip(), flags=0).group(1)
#     print(bid_no)

In [30]:
#招標決標資料塞進DB
def send_db(raw_datas,data_class, db_settings,Not_update_detail=True):
    errMsg=None
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
      #資料表相關操作
        get_max_log_id=f"select MAX(UID) FROM {db_settings['db']}.log"
        # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
        if Not_update_detail and (data_class =='tenderDeclaration' or data_class =='final' or data_class =='searchAppeal'):
            command = f"INSERT INTO {db_settings['db']}.gov_purchase(log_UID, proposer_name, bid_no, bid_name, times, typ, clas, date, enddate, budget, ischange, detail_connect, detail_data, detail_data_html, get_data_date, tenderType,error_code)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),get_data_date=values(get_data_date),times =values(times),typ=values(typ),clas=values(clas),date = values(date),enddate=values(enddate),budget=values(budget),ischange=values(ischange),detail_connect=values(detail_connect)"
            db_class = "api_process_list"
        else:
            command = f"INSERT INTO {db_settings['db']}.gov_purchase(log_UID, proposer_name, bid_no, bid_name, times, typ, clas, date, enddate, budget, ischange, detail_connect, detail_data, detail_data_html, get_data_date, tenderType,error_code)VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),get_data_date=values(get_data_date),times =values(times),typ=values(typ),clas=values(clas),date = values(date),enddate=values(enddate),budget=values(budget),ischange=values(ischange),detail_connect=values(detail_connect),detail_data=values(detail_data),detail_data_html=values(detail_data_html),error_code=values(error_code)"
            db_class = "api_process_detail"
        command_log = f"INSERT INTO {db_settings['db']}.log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
        # 紀錄開始 (暫時取消)
#         cursor.execute(command_log, (datetime.datetime.now(), "gov_purchase", "start", data_class, ""))
        # 取得 log 的 UID
        cursor.execute(get_max_log_id)
        log_UID = str(cursor.fetchone()[0])
        # 空資料處理
        if len(raw_datas)==0:
            #cursor.execute(command_log, (datetime.datetime.now(), db_class, "wrong", data_class, "無資料可爬取"))
            pass
        elif type(raw_datas) == dict:
            data_combine.append((int(log_UID), raw_datas["proposer_name"], raw_datas["bid_no"], raw_datas["bid_name"], raw_datas["times"], raw_datas["typ"], raw_datas["clas"], raw_datas["date"], raw_datas["enddate"], raw_datas["budget"], raw_datas["ischange"], raw_datas["detail_connect"], json.dumps(raw_datas["detail_data"],ensure_ascii=False), raw_datas["detail_data_html"],raw_datas["get_data_date"], raw_datas["tenderType"],raw_datas["error_code"]))
        # 組合數據
        else:
            #抓取到的資料是以公告日期最新到最舊，這樣寫進資料庫時最新的資料會被舊的資料覆蓋住，所以加一個reversed，讓最舊的先進去
            for raw_data in reversed(raw_datas):
                data_combine.append((int(log_UID), raw_data["proposer_name"], raw_data["bid_no"], raw_data["bid_name"], raw_data["times"], raw_data["typ"], raw_data["clas"], raw_data["date"], raw_data["enddate"], raw_data["budget"], raw_data["ischange"], raw_data["detail_connect"], json.dumps(raw_data["detail_data"],ensure_ascii=False), raw_data["detail_data_html"],raw_data["get_data_date"], raw_data["tenderType"],raw_data["error_code"]))  # 注意要用两个括号扩起来

            # 執行
            try:
                cursor.executemany(command, data_combine)
                time_end = datetime.datetime.now()    #結束計時
                time_c= time_end - time_start   #執行所花時間
                print('招標決標資料塞進DB，time cost', time_c, 's')
            except Exception as ex:
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print("send_db" + errMsg)
                raise
                cursor.execute(command_log, (datetime.datetime.now(), "fail", "wrong", data_class, str(error_class)+ "  " + str(detail) + errMsg))
        # 紀錄結束
        if not errMsg:
            cursor.execute(command_log, (datetime.datetime.now(), db_class, "success", data_class, str(len(raw_datas)) + " 筆資料已完成"))


In [31]:
# from collections import Counter   #引入Counter
# a = ll
# b = dict(Counter(a))
# print ([key for key,value in b.items()if value > 1])  #只展示重複元素

In [32]:
# global waitt
# waitt =0
# x5 = gov_serach_crawler("searchAppeal", headers = headers,day_before=0)
# send_db(x5,"searchAppeal", db_settings)

In [33]:
#tenderDeclaration
#searchAppeal
#publicRead
#final
# global waitt
# waitt =0
# for i in range(1,100):
#     x5 = gov_serach_crawler("final", headers = headers,day_before=i)
#     send_db(x5,"final", db_settings)
#     x5 = gov_serach_crawler("tenderDeclaration", headers = headers,day_before=i)
#     send_db(x5,"tenderDeclaration", db_settings)
#     x5 = gov_serach_crawler("searchAppeal", headers = headers,day_before=i)
#     send_db(x5,"searchAppeal", db_settings)
#     x5 = gov_serach_crawler("publicRead", headers = headers,day_before=i)
#     send_db(x5,"publicRead", db_settings)

In [34]:
# s = rq.session()
# ii=1
# s.keep_alive = True
# headers["User-Agent"] = UserAgent(use_cache_server=False).random
# count_per_page = 10
# get_data_date = (datetime.date.today() - datetime.timedelta(days=0))
# current_date = get_data_date.strftime("%Y/%m/%d")
# current_date = str(int(current_date[0:4])) + current_date[4:]
# url_first_init = 'https://t2.pcc.gov.tw/prkms/tpAppeal/common/readTpAppeal/basic/returnToBasic'
# data={'pageSize':str(count_per_page),'firstSearch':'true','searchType':'basic','level_1':'on','tenderType':'SEARCH_APPEAL','dateType':'isDate','endDate':current_date,'startDate':current_date}
# data = urlencode(data)
# url_first = f"{url_first_init}?{data}"
# url = f"{url_first}&d-4025577-p="
# global progress_searchAppeal
# r = s.get(url_first, headers = headers)
# raw_datas=[]
# r = s.get(url + str(1), headers = headers)
# html_doc = r.text
# if re.search(r'無符合條件資料',html_doc):
#     print("無符合條件資料")
# soup = BeautifulSoup(html_doc, parser)
        
# data = soup.select('table#tpAppeal')
# all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip())
# xxxx = 0
# print(data)
# for i in data[0].select('tr'):
#     xxxx = xxxx + 1
#     progress_searchAppeal = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxx)/all_data*100) + " %。"
#     print(progress_searchAppeal)
#     # 取得資料日期

#     # 項次
#     no=i.select('td')[0].text.strip()
#     # 機關名稱
#     proposer_name=i.select('td')[1].text.strip()
#     # 標案案號
#     bid_no=re.match(r'<script>var hw = Geps3.CNS.pageCode2Img\(\"(.*)\"\);',str(i.select('td')[3].select('span script')[0]), flags=0).group(1)
#     # 是否更正
#     ischange=0  
#     # 標案名稱    
#     bid_name=i.select('td')[3].text.strip()
#     # 傳輸次數 (公告次數)
#     times=i.select('td')[4].text.strip()
#     # 招標方式
#     typ=""
#     # 採購性質
#     clas=""
#     # 公告日期
#     date=i.select('td')[5].text.strip()[0:9]
#     # 截止投標
#     enddate=i.select('td')[5].text.strip()[-9:]
#     # 預算金額
#     budget=""
#     # 內文連結
#     detail_connect = "https://t2.pcc.gov.tw" + i.select('td')[6].select('a')[0]['href']
#     (detail_data,detail_data_html) = get_searchAppeal_detail(headers,detail_connect)
#     raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_data':detail_data, 'detail_connect':detail_connect,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"searchAppeal",'error_code':0})


In [35]:
def gov_serach_crawler(url_type,headers,start_page=1,day_before=0):
    global waitt
    get_data_date = (datetime.date.today() - datetime.timedelta(days=day_before))
    current_date = get_data_date.strftime("%Y/%m/%d")
    current_date = str(int(current_date[0:4])) + current_date[4:]
    # 取得資料日期
    print("！！！！！爬取日期："+current_date)
    s = rq.session()
    s.keep_alive = True
    headers["User-Agent"] = UserAgent(use_cache_server=False).random
    
    count_per_page = 100
    host = 'web.pcc.gov.tw'
    headers["Host"]=host
    domain = 'https://' + host
    
    if url_type == "tenderDeclaration":
        url_first_init = f'{domain}/prkms/tender/common/basic/readTenderBasic'
        data={'pageSize':str(count_per_page),'firstSearch':'true','searchType':'basic','level_1':'on','tenderType':'TENDER_DECLARATION','tenderWay':'TENDER_WAY_ALL_DECLARATION','dateType':'isDate','tenderEndDate':current_date,'tenderStartDate':current_date}
        data = urlencode(data)
        url_first = f"{url_first_init}?{data}"
        url = f"{url_first}&d-49738-p="
        global progress_tenderDeclaration
    elif url_type == "searchAppeal":  
        url_first_init = f'{domain}/prkms/tpAppeal/common/readTpAppeal/basic/returnToBasic'
        data={'pageSize':str(count_per_page),'firstSearch':'true','searchType':'basic','level_1':'on','tenderType':'SEARCH_APPEAL','dateType':'isDate','endDate':current_date,'startDate':current_date}
        data = urlencode(data)
        url_first = f"{url_first_init}?{data}"
        url = f"{url_first}&d-4025577-p="
        global progress_searchAppeal
    elif url_type == "publicRead":
        url_first_init = f'{domain}/prkms/tpRead/common/readTpRead/basic/returnToBasic'
        data={'pageSize':str(count_per_page),'firstSearch':'true','searchType':'basic','level_1':'on','tenderType':'PUBLIC_READ','dateType':'isDate','queryEndDate':current_date,'queryStartDate':current_date}
        data = urlencode(data)
        url_first = f"{url_first_init}?{data}"
        url = f"{url_first}&d-447556-p="
        global progress_publicRead
    elif url_type == "predict":
        url_first_init = f'{domain}/prkms/gpaPredict/common/readGpaPredict/basic/returnToBasic'
        data={'pageSize':str(count_per_page),'firstSearch':'true','searchType':'basic','level_1':'on','tenderType':'PREDICT','dateType':'isDate','predictNoticeDateEnd':current_date,'predictNoticeDateStart':current_date}
        data = urlencode(data)
        url_first = f"{url_first_init}?{data}"
        url = f"{url_first}&d-49738-p="
        global progress_predict
    elif url_type == "final":  
        url_first_init = f'{domain}/prkms/tender/common/agent/readTenderAgent'
        data={'pageSize':str(count_per_page),'firstSearch':'false','tenderStatus':'TENDER_STATUS_1','tenderWay':'TENDER_WAY_ALL_DECLARATION','tenderRange':'TENDER_RANGE_ALL','awardAnnounceEndDate':current_date,'awardAnnounceStartDate':current_date}
        data = urlencode(data)
        url_first = f"{url_first_init}?{data}"
        url = f"{url_first}&d-16396-p="
        global progress_final
    else:
        raise Exception("url_type 輸入錯誤") 

    while waitt:
            print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖")
            time.sleep(60*waitt)
            
    while True:
        try:
            r = s.get(url_first, headers = headers)
            if r.status_code<300:
                waitt = 0
                break
        except Exception as ex: 
            waitt = waitt + 1
            print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖")
            print(ex)
            time.sleep(1200*waitt)


    print("外頁進入點存活")  
    raw_datas=[]
    for ii in range(start_page,2000):
        while waitt:
                print(url_type + f"暫停休息 {60*waitt}秒 一下，有執行緒被鎖_phase2")
                time.sleep(60*waitt)
        while True:
            try:
                r = s.get(url + str(ii), headers = headers)
                print("外頁列表頁存活")  
                if r.status_code == 200:
                    waitt = 0
                    break
            except Exception as ex: 
                print(url_type + f"暫停休息 {1200*waitt}秒 一下，本機 IP 被鎖_phase2")
                print(ex)
                waitt = waitt + 1
                time.sleep(1200*waitt)
        print("頁面前往："+str(r.url))
        html_doc = r.text
        if re.search(r'無符合條件資料',html_doc):
            print("無符合條件資料")
            break
        soup = BeautifulSoup(html_doc, parser)
        
        time_start = datetime.datetime.now()
        if url_type == "tenderDeclaration":
            data = soup.select('div#printArea table#tpam')
            thead_tag = data[0].find('thead')
            if thead_tag:
                thead_tag.decompose()
            all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip().replace(',',''))
            xxx = 0
            for i in data[0].select('tr'):
                t_s = datetime.datetime.now()
                xxx = xxx + 1
                progress_tenderDeclaration = "這是第 " + str(ii) + " 頁，的第 "+ str(xxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxx)/all_data*100) + " %。"
                print(progress_tenderDeclaration)
                # 取得資料日期

                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                if i.select('td')[2].select('font'):
                    ischange=1
                else:
                    ischange=0  
                # 標案名稱    
                bid_name=re.match(r'.*\(\"(<.*>)*(.*)\"\);',str(i.select('td')[2].select('a script')[0]), flags=0).group(2)
                # 傳輸次數
                times=i.select('td')[3].text.strip()
                # 招標方式
                typ=i.select('td')[4].text.strip()
                # 採購性質
                clas=i.select('td')[5].text.strip()
                # 公告日期
                date=i.select('td')[6].text.strip()
                # 截止投標
                enddate=i.select('td')[7].text.strip()
                # 預算金額
                budget=i.select('td')[8].text.strip()
                # 內文連結
                detail_connect = domain + i.select('td')[2].select('a')[0]['href']
                detail_data ={}
                detail_data_html=""
                t_e = datetime.datetime.now()
                time_se= t_e - t_s
                print('此細項已抓取完成，總耗時：', time_se, 's')
                print("------------------------------------")
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"tenderDeclaration",'error_code':0})
        elif url_type == "searchAppeal":
            data = soup.select('table#tpAppeal')
            thead_tag = data[0].find('thead')
            if thead_tag:
                thead_tag.decompose()
            all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip().replace(',',''))
            xxxx = 0
            for i in data[0].select('tr'):
                xxxx = xxxx + 1
                progress_searchAppeal = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxx)/all_data*100) + " %。"
                print(progress_searchAppeal)
                # 取得資料日期

                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=re.match(r'.*\(\"(<.*>)*(.*)\"\);',str(i.select('td')[3].select('span script')[0]), flags=0).group(2)
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=i.select('td')[5].text.strip()[0:9]
                # 截止投標
                enddate=i.select('td')[5].text.strip()[-9:]
                # 預算金額
                budget=""
                # 內文連結
                detail_connect = domain + i.select('td')[6].select('a')[0]['href']
                detail_data={}
                detail_data_html=''
                #(detail_data,detail_data_html) = get_searchAppeal_detail(headers,detail_connect)
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_data':detail_data, 'detail_connect':detail_connect,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"searchAppeal",'error_code':0})
        elif url_type == "publicRead":
            data = soup.select('table#tpRead')
            thead_tag = data[0].find('thead')
            if thead_tag:
                thead_tag.decompose()
            all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip().replace(',',''))
            xxxxx=0
            for i in data[0].select('tr'):
                xxxxx = xxxxx + 1
                progress_publicRead = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " +str(((ii-1)*count_per_page+xxxxx)/all_data*100) + " %。"
                print(progress_publicRead)
                # 取得資料日期

                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[2].text.strip()
                # 內文連結
                detail_connect = domain + i.select('td')[6].select('a')[0]['href']
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=re.match(r'.*\(\"(<.*>)*(.*)\"\);',str(i.select('td')[3].select('span script')[0]), flags=0).group(2)
                # 傳輸次數 (公告次數)
                times=i.select('td')[4].text.strip()
                # 招標方式
                typ=""
                # 採購性質
                clas=""
                # 公告日期
                date=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(1).strip()
                # 截止投標
                enddate=re.match(r'(.*)\s*─\s*(.*)', i.select('td')[5].text.strip(), flags=0).group(2).strip()
                # 預算金額
                budget=""
                (detail_data,detail_data_html)=get_publicRead_detail(headers,detail_connect)
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"publicRead",'error_code':0})
        elif url_type == "predict" : 
            data = soup.select('table#gpaPredict')
            thead_tag = data[0].find('thead')
            if thead_tag:
                thead_tag.decompose()
            all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip().replace(',',''))
            xxxxx = 0
            for i in data[0].select('tr'):
                xxxxx = xxxxx + 1
                progress_predict = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxxx)/all_data*100) + " %。"
                print(progress_predict)
                # 取得資料日期

                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[1].text.strip()
                # 是否更正
                ischange=0  
                # 標案名稱    
                bid_name=i.select('td')[2].select('a')[0].text.strip()
                # 傳輸次數
                times=""
                # 招標方式
                typ=i.select('td')[4].text.strip()
                # 採購性質
                clas=""
                # 公告日期
                date=i.select('td')[5].text.strip()
                # 截止投標
                enddate=""
                # 預算金額
                budget=""
                # 內文連結
                detail_connect = domain + i.select('td')[6].select('a')[0]['href'][5:]
                print(detail_connect)
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"predict",'error_code':0})
        elif url_type == "final" : 
            data = soup.select('table#atm')
            thead_tag = data[0].find('thead')
            if thead_tag:
                thead_tag.decompose()
            all_data = int(soup.select('div#displaytagBannerDiv.page span#pagebanner span')[0].text.strip().replace(',',''))
            xxxxxx = 0
            for i in data[0].select('tr'):
                t_s = datetime.datetime.now()
                xxxxxx = xxxxxx + 1
                progress_final = "這是第 " + str(ii) +" 頁，的第 "+ str(xxxxxx) + " 項，總共有 " + str(all_data) + " 項，目前進度是 " + str(((ii-1)*count_per_page+xxxxxx)/all_data*100) + " %。"
                print(progress_final)
                # 取得資料日期

                # 項次
                no=i.select('td')[0].text.strip()
                # 機關名稱
                proposer_name=i.select('td')[1].text.strip()
                # 標案案號
                bid_no=i.select('td')[2].text.strip()
                # 是否更正
                if i.select('td')[2].select('font'):
                    ischange=1
                else:
                    ischange=0   
                # 標案名稱    
                bid_name=re.match(r'.*\(\"(<.*>)*(.*)\"\);',str(i.select('td')[2].select('a script')[0]), flags=0).group(2)
                # 傳輸次數
                times=i.select('td')[8].text.strip()
                # 這樣可以從內頁資料抓傳輸次數： 上面有定義 dicMemberCheck 可以吐出字典裡對應key的value
#                 if detail_data['ischange'] ==0 and not re.search(r'限制性招標',detail_data['招標方式']) and not re.search(r'選擇性招標',detail_data['招標方式']):
#                     print(detail_data['新增公告傳輸次數'])
                
                # 招標方式
                typ=i.select('td')[3].text.strip()
                # 採購性質
                clas=i.select('td')[4].text.strip()
                # 公告日期
                date=i.select('td')[5].text.strip()
                # 截止投標
                enddate=""
                # 預算金額
                budget=i.select('td')[6].text.strip()
                # 內文連結
                detail_connect = domain + i.select('td')[2].select('a')[0]['href']
                detail_data={}
                detail_data_html=""
                t_e = datetime.datetime.now()
                time_se= t_e - t_s
                print('此細項已抓取完成，總耗時：', time_se, 's')
                print("------------------------------------")
                raw_datas.append({'no': no,'proposer_name': proposer_name,'bid_no': bid_no,'bid_name': bid_name,'times': times,'typ': typ,'clas': clas,'date': date,'enddate': enddate,'budget': budget,'ischange': ischange,'detail_connect':detail_connect,'detail_data':detail_data,'detail_data_html':detail_data_html,'get_data_date':get_data_date,'tenderType':"final",'error_code':0})
        time_end = datetime.datetime.now()
        time_c= time_end - time_start
        if ii*count_per_page >= all_data:
            print(f'此分頁抓取總耗時：{time_c} s，已完成 {all_data} 筆，共有 {all_data} 筆')
            if url_type == "tenderDeclaration":
                del progress_tenderDeclaration
            elif url_type == "searchAppeal":
                del progress_searchAppeal
            elif url_type == "publicRead":    
                del progress_publicRead
            elif url_type == "predict":    
                del progress_predict
            elif url_type == "final":    
                del progress_final
            break
        else:
            print(f'此分頁抓取總耗時：{time_c} s，已完成 {ii*count_per_page} 筆，共有 {all_data} 筆')
    return raw_datas

In [36]:
def search_by_date(s,start_date, end_date,if_publish,headers):
    r = s.get("https://web.pcc.gov.tw/prkms/prms-viewDailyTenderListClient.do?root=tps",headers=headers)
    html_doc = r.text
    if r.status_code == 500 or re.search(r'The server encountered an internal error or misconfiguration and was unable to complete your request',html_doc):
        raise Exception("打不進去header頁") 
        return ""
    soup = BeautifulSoup(html_doc, parser)
    publishs_href=[]
    publishs_date=[]
    not_publishs_href=[]
    not_publishs_date=[]
    left = soup.select('td#page table tr td:nth-child(2) li a')
    right = soup.select('td#page table tr td:nth-child(4) li a')
    for i in left:
        publishs_href.append(i['href'])
        y=str(int(re.match(r'(.*)年(.*)月(.*)日',i['title']).group(1))+1911)
        m=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(2)
        d=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(3)
        ymd = datetime.datetime.strptime("-".join([y,m,d]), "%Y-%m-%d")
        publishs_date.append(ymd)
    for i in right:
        not_publishs_href.append(i['href'])
        y=str(int(re.match(r'(.*)年(.*)月(.*)日',i['title']).group(1))+1911)
        m=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(2)
        d=re.match(r'(.*)年(.*)月(.*)日',i['title']).group(3)
        ymd = datetime.datetime.strptime("-".join([y,m,d]), "%Y-%m-%d")
        not_publishs_date.append(ymd)
    for i in publishs_date:
        publishs_start_date_index = publishs_date.index(i)
        if i < start_date:
            break
    for i in not_publishs_date:
        not_publishs_start_date_index = not_publishs_date.index(i)
        if i < start_date:
            break
    for i in publishs_date:
        publishs_end_date_index = publishs_date.index(i)
        if i <= end_date:
            break
    for i in not_publishs_date:
        not_publishs_end_date_index = not_publishs_date.index(i)
        if i <= end_date:
            break

    if not_publishs_start_date_index<not_publishs_end_date_index:
        raise Exception("開始日期不可大於結束日期")
    if publishs_start_date_index<publishs_end_date_index:
        raise Exception("開始日期不可大於結束日期")
    if start_date > not_publishs_date[0] or start_date > publishs_date[0]:
        raise Exception("無資料可爬，初始日期大於網站最新日期")
    
    total_hrefs=[]
    if if_publish=="all" or if_publish=="yes":
        for i in range(0,len(publishs_href[publishs_end_date_index:publishs_start_date_index])):
            total_href={'href':publishs_href[publishs_end_date_index+i],'date':publishs_date[publishs_end_date_index+i],'ispublish':1}
            total_hrefs.append(total_href)
    if if_publish=="all" or if_publish=="no":
        for i in range(0,len(not_publishs_href[not_publishs_end_date_index:not_publishs_start_date_index])):
            total_href={'href':not_publishs_href[not_publishs_end_date_index+i],'date':not_publishs_date[not_publishs_end_date_index+i],'ispublish':0}
            total_hrefs.append(total_href)
    return total_hrefs

In [37]:
def date_serach_crawler(s,search_date,headers):
    url_head = "https://web.pcc.gov.tw/prkms/"
    len_search_date=len(search_date)
    global progress_date_serach
    if len_search_date==0:
        return
    xxxxxx = 0
    for i in search_date:
        xxxxxx = xxxxxx + 1
        progress_date_serach = "這是第 " + str(xxxxxx) + " 個，總共有 " + str(len_search_date) + " 個要爬"
        print(search_date.index(i))
        r = s.get(url_head + i['href'], headers=headers)
        html_doc = r.text
        soup = BeautifulSoup(html_doc, parser)
        i['detail_data'] = soup.select('td#page table table table table tr td')[0]
    del progress_date_serach
    return search_date

In [38]:
def send_db_2(raw_datas,data_class, db_settings):
    time_start = datetime.datetime.now() #開始計時
    data_combine=[]
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            get_max_log_id=f"select MAX(UID) FROM {db_settings['db']}.log"
            # % 操作符只能直接用於字串(‘123’)，列表([1,2,3])、元組
            command = f"INSERT INTO {db_settings['db']}.date_search_by_date(log_UID, date, ispublish, upper, lower, href)VALUES(%s, %s, %s, %s, %s, %s) on DUPLICATE KEY UPDATE log_UID = values(log_UID),date=values(date),ispublish =values(ispublish),upper=values(upper),lower=values(lower),href = values(href)"
            command_log = f"INSERT INTO {db_settings['db']}.log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始 (暫時取消)
#             cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "start", data_class, ""))
            # 取得 log 的 UID
            cursor.execute(get_max_log_id)
            log_UID = str(cursor.fetchone()[0])
            # 組合數據
            for raw_data in raw_datas:
                data_combine.append((int(log_UID), raw_data["date"], raw_data["ispublish"], "", raw_data["detail_data"], raw_data["href"]))  # 注意要用两个括号扩起来

            # 執行
            try:
                cursor.executemany(command, data_combine)
            except Exception as ex:
                print(ex)
                error_class = ex.__class__.__name__ #取得錯誤類型
                detail = ex.args[0] #取得詳細內容
                cl, exc, tb = sys.exc_info() #取得Call Stack
                lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
                fileName = lastCallStack[0] #取得發生的檔案名稱
                lineNum = lastCallStack[1] #取得發生的行號
                funcName = lastCallStack[2] #取得發生的函數名稱
                errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
                print(errMsg)
                cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "fail", data_class, error_class + detail + errMsg))
            # 紀錄結束
            cursor.execute(command_log, (datetime.datetime.now(), "date_search_by_date", "success", data_class, str(len(raw_datas)) + " 筆資料已完成"))

    except Exception as ex:
        print('send_db_2 出現錯誤：' + str(ex))

    time_end = datetime.datetime.now()    #結束計時
    time_c= time_end - time_start   #執行所花時間
    print('time cost', time_c, 's')

In [ ]:
#雙重 thread
my_host=config_my_host
print(f"my_host：{my_host}")
app = Flask(__name__)

command_log = f"INSERT INTO {db_settings['db']}.log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"

@app.route('/', methods=['GET'])
def test():
#     ip = request.remote_addr
#     print(ip)
#     conn = pymysql.connect(**db_settings)
#     # 建立Cursor物件
#     with conn.cursor() as cursor:
#         if_finish_command="SELECT status FROM log where task ='api_process_list' order by UID DESC"
#         cursor.execute(if_finish_command)
#         if cursor.fetchone() is None:
#             cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "initial", "", ""))
#         cursor.execute(if_finish_command)
#         if_finish = str(cursor.fetchone()[0])
#     if re.search(r'finish', if_finish, flags=0) or if_finish == "initial":
#         wording ='<p>恭喜!網站建置成功!</p><p>api說明：</p><p>1.&nbsp;/thread：查看當前執行緒狀態，無須參數 (get請求)</p><p>2.&nbsp;/start_task_all：開始執行爬蟲，若要強制重啟，參數帶 restart = 1 (get請求)</p><p>3.&nbsp;/date_search_crawler：公告日期執行爬蟲，參數帶 start_date、end_date、if_publish (get請求)<br />start_date、end_date =&gt; 以字串型態帶日期，ex:20210125，if_publish =&gt; 以字串形式帶 all | yes | no </p><p>http://127.0.0.1:5000/date_search_crawler?start_date=20210128&end_date=20210204&if_publish=yes</p><p>4.&nbsp;/progress：查看當前執行緒進度，無須參數 (get請求)</p><p>5. /proxy : 從免費代理網站抓取proxy進資料庫，無須參數 (get請求)'
#         status_code = 200
#     else:
#         wording ="I am busy now, please wait"
#         status_code = 500
        
    wording ='<p>恭喜!網站建置成功!</p><p>api說明：</p><p>1.&nbsp;/thread：查看當前執行緒狀態，無須參數 (get請求)</p><p>2.&nbsp;/start_task_list：執行爬蟲：抓列表資料，若要強制重啟，參數帶 restart = 1；若要設定爬取的初始頁面，參數帶 start_page=[x,x,x,x] ([招標公告頁數、公開徵求頁數、公開閱覽頁數、決標公告頁數])；若想要設定爬取 N 天前的資料，參數要帶 day_before，以 int 的方式放值，ex: day_before=0  (get請求)</p><p>http://127.0.0.1:5000/start_task_list?restart=1&start_page=[1,1,1,1]&day_before=0</p><p><p>3.&nbsp;/start_task_detail：執行爬蟲：抓內頁資料，要晚於 /start_task_list 5分鐘，spilt 參數代表分段執行，spilt[0]是第幾段；spilt[1]是全部共幾段 (get請求)</p><p>http://127.0.0.1:5000/start_task_detail?spilt=(1,2)</p>4.&nbsp;/date_search_crawler：執行爬蟲：依公告日期查詢，參數帶 start_date、end_date、if_publish (get請求)<br />start_date、end_date =&gt; 以字串型態帶日期，ex:20210125，if_publish =&gt; 以字串形式帶 all | yes | no </p><p>http://127.0.0.1:5000/date_search_crawler?start_date=20210128&end_date=20210204&if_publish=yes</p><p>5.&nbsp;/progress：查看當前執行緒進度，無須參數 (get請求)</p><p>6. /proxy : 從免費代理網站抓取proxy進資料庫，無須參數 (get請求)' 
    status_code = 200

    return wording,status_code

@app.route('/thread', methods=['GET'])
def thread_status():
    wording =""
    wording2 =""
    wording3 =""
    wording4 =""
    wording5=""
    wording6=""
    code =200
    print(str(threading.enumerate()))
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_searchAppeal") >=0:
            wording = "當前正在執行 thread_tenderDeclaration_searchAppeal 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_date_search_crawler") >=0:
            wording2 = "當前正在執行 thread_date_search_crawler 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_proxy_crawler") >=0:
            wording3 = "當前正在執行 thread_proxy_crawler 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_datesearch_publicRead") >=0:
            wording4 = "當前正在執行 thread_final_datesearch_publicRead 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_detail") >=0:
            wording5 = "當前正在執行 thread_tenderDeclaration_detail 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_detail") >=0:
            wording6 = "當前正在執行 thread_final_detail 執行緒"
            code = 500
    except Exception as ex:
        print(ex)
        pass
        
    return wording + " " + wording2 + " " + wording3 + " " + wording4 + " " + wording5 + " " + wording6,code
        
@app.route('/proxy', methods=['GET'])
def proxy():
    def proxyy():
        p_yes=get_proxy_ip("yes",db_settings)
        #p_no=get_proxy_ip("no",db_settings)
        send_ip(p_yes, db_settings ,"only https",new = True)
        #send_ip(p_no, db_settings ,"only http",new = True)
    try:
        if str(threading.enumerate()).find("thread_proxy_crawler") >=0:
            return "已經在爬 proxy 了，別吵",500
    except Exception as ex:
        print(ex)
        pass    
    
    thread_proxy_crawler = threading.Thread(target=proxyy, kwargs={},name="thread_proxy_crawler")
    thread_proxy_crawler.start()
    return "started",200
    
@app.route('/start_task_list', methods=['GET'])
def start_task_all():
    if request.args:
        dd= request.args.to_dict()
        if set(['start_page','restart','day_before']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' start_page '+'restart '+'day_before',500
    def do_work(start_page,day_before,db_settings,headers):
        get_data_date = (datetime.date.today() - datetime.timedelta(days=day_before))
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " start", "tenderDeclaration_searchAppeal", f"爬取日期：{get_data_date}，tenderDeclaration 從第 {start_page[0]} 頁開始爬，searchAppeal 從第 {start_page[1]} 頁開始爬"))
        time_start = time.time()
        print("=====1=======")
        
        try:
            x1 = gov_serach_crawler("tenderDeclaration", headers = headers,start_page = int(start_page[0]),day_before=day_before)
            send_db(x1,"tenderDeclaration", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "tenderDeclaration", errMsg))
        print("=====2=======")
        try:
            x2 = gov_serach_crawler("searchAppeal", headers = headers,start_page = int(start_page[1]),day_before=day_before)
            send_db(x2,"searchAppeal", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "searchAppeal", errMsg))
        
#         try:
#             x4 = gov_serach_crawler("predict",proxies,db_settings, headers = headers, account=account,password=password)
#             send_db(x4,"predict", db_settings)
#         except Exception as ex:
#             print(ex)
#             error_class = ex.__class__.__name__ #取得錯誤類型
#             detail = ex.args[0] #取得詳細內容
#             cl, exc, tb = sys.exc_info() #取得Call Stack
#             lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
#             fileName = lastCallStack[0] #取得發生的檔案名稱
#             lineNum = lastCallStack[1] #取得發生的行號
#             funcName = lastCallStack[2] #取得發生的函數名稱
#             errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
#             print(errMsg)
#             conn = pymysql.connect(**db_settings)
#             with conn.cursor() as cursor:
#                 cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "predict", errMsg))
        print("=====4=======")
        time_d = time.time()
        time_c = time_d - time_start
        print("！！！！！  thread_tenderDeclaration_searchAppeal 總執行時間 "+str(time_c) +" 秒")
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " finish", "tenderDeclaration_searchAppeal", ""))
            
    def do_work_final(start_page,day_before,db_settings,headers):
        get_data_date = (datetime.date.today() - datetime.timedelta(days=day_before))
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " start", "final_publicRead_datesearch", f"爬取日期：{get_data_date}，final 從第 {start_page[3]} 頁開始爬，publicRead 從第 {start_page[2]} 頁開始爬"))
        time_start = time.time()
        try:
            time.sleep(5)
            x5 = gov_serach_crawler("final", headers = headers,start_page = int(start_page[3]),day_before=day_before)
            send_db(x5,"final", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "final", errMsg))
        print("=====3=======")        
        try:
            x3 = gov_serach_crawler("publicRead", headers = headers,start_page = int(start_page[3]),day_before=day_before)
            send_db(x3,"publicRead", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "publicRead", errMsg))
#依公告日期招標
#         try:
#             s = rq.session()
#             xx = search_by_date(s,datetime.datetime.today()- datetime.timedelta(days=1), datetime.datetime.today(),"all", headers = headers)
#             print(xx)
#             xxx = date_search_crawler(s,xx,headers)
#             send_db_2(xxx,"date_search_by_date", db_settings)
#         except Exception as ex:
#             print(ex)
#             error_class = ex.__class__.__name__ #取得錯誤類型
#             detail = ex.args[0] #取得詳細內容
#             cl, exc, tb = sys.exc_info() #取得Call Stack
#             lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
#             fileName = lastCallStack[0] #取得發生的檔案名稱
#             lineNum = lastCallStack[1] #取得發生的行號
#             funcName = lastCallStack[2] #取得發生的函數名稱
#             errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
#             print(errMsg)
#             conn = pymysql.connect(**db_settings)
#             with conn.cursor() as cursor:
#                 cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "date_search_crawler", str(detail)))
      
        time_d = time.time()
        time_c = time_d - time_start
        print("！！！！！  thread_final_datesearch_publicRead 總執行時間 "+str(time_c) +" 秒")
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " finish", "final_publicRead_datesearch", ""))
    
    #資料庫log檢查
    ip = request.remote_addr
    conn = pymysql.connect(**db_settings)
    if_finish_command=f"SELECT status FROM {db_settings['db']}.log where task ='api_process_list' order by UID DESC"
    with conn.cursor() as cursor:
        cursor.execute(if_finish_command)
        if cursor.fetchone() is None:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "initial", "", ""))
        cursor.execute(if_finish_command)
        if_finish = str(cursor.fetchone()[0])
    
    # 不判斷資料庫log強制重啟
    if re.search(r'finish', if_finish, flags=0) or re.search(r'wrong', if_finish, flags=0) or if_finish == "initial":
        pass
    elif "restart" in request.args:
        try:
            if request.args.get("restart")==1:
                pass
        except:
            return "若想要重啟爬蟲，參數要帶restart，值請放 1 "
    else:
        return "資料庫未收到結束log，可帶參數 restart=1 忽略資料庫log判斷",500
    #執行序之間任一ip被鎖block全部
    global waitt
    waitt = 0
    
    day_before=0
    if 'day_before' in request.args:
        try:
            day_before = eval(request.args.get("day_before"))
        except:
            return "若想要設定爬取 N 天前的資料，參數要帶 day_before，以 int 的方式放值，ex: day_before=0 "

    
    #設定起始頁面
    start_page=[1,1,1,1]
    if 'start_page' in request.args:
        try:
            start_page = eval(request.args.get("start_page"))
            if len(start_page) !=4:
                return "start_page 參數長度不足 4 (須設定 4 個：tenderDeclaration、searchAppeal、publicRead、final)，無須爬取的項目可輸入1999！"
            else:
                print("頁數參數接收成功")
        except:
            return "若想要設定爬取的初始頁數，參數要帶 start_page，以 list 的方式放值=>[招標公告頁數,公開徵求頁數,公開閱覽頁數,決標公告頁數]，ex: start_page=[1,1,1,1] "
    print(f"tenderDeclaration 從第 {start_page[0]} 頁開始爬取")
    print(f"searchAppeal 從第 {start_page[1]} 頁開始爬取")
    print(f"publicRead 從第 {start_page[2]} 頁開始爬取")
    print(f"final 從第 {start_page[3]} 頁開始爬取")
    
    try:
        if str(threading.enumerate()).find("thread_tenderDeclaration_searchAppeal") >= 0 :
            wording0="thread_tenderDeclaration_searchAppeal 執行緒正在執行；"
        else:
            thread_tenderDeclaration_searchAppeal = threading.Thread(target=do_work, kwargs={'start_page':start_page,'day_before':day_before,'db_settings': db_settings,'headers': headers},name="thread_tenderDeclaration_searchAppeal")
            thread_tenderDeclaration_searchAppeal.start()
            wording0="啟動 thread_tenderDeclaration_searchAppeal 執行緒；"
    except Exception as ex:
        print(ex)
        pass
    
    try:
        if str(threading.enumerate()).find("thread_final_datesearch_publicRead") >= 0 :
            wording1="thread_final_datesearch_publicRead 執行緒正在執行"
        else:
            thread_final_datesearch_publicRead = threading.Thread(target=do_work_final, kwargs={'start_page':start_page,'day_before':day_before,'db_settings': db_settings,'headers': headers},name="thread_final_datesearch_publicRead")
            thread_final_datesearch_publicRead.start()
            wording1="啟動 thread_final_datesearch_publicRead 執行緒"
    except Exception as ex:
        print(ex)
        pass

    return wording0+wording1,200

@app.route('/start_task_detail', methods=['GET'])
def start_task_detail():
    path = config_path
    if os.path.exists(path):
        shutil.rmtree (path)
        os.mkdir(path)
    else:
        os.mkdir(path)
    ip = request.remote_addr

    def do_work_detail(tenderType,spilt,diff_seconds_data,diff_seconds_conn,db_settings,headers,No_proxy,block_crawl_no,path):
        if tenderType == "final":
            global progress_final_detail
            progress_final_detail =''
        elif tenderType =="tenderDeclaration":
            global progress_tenderDeclaration_detail
            progress_tenderDeclaration_detail =''
        
        conn = pymysql.connect(**db_settings)
        get_data=f"SELECT * FROM {db_settings['db']}.gov_purchase " + " where (detail_data_html ='' and detail_data like '%{}%') and error_code = 0  and tenderType='"+tenderType+"'"
        # 建立Cursor物件
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
          #資料表相關操作
            cursor.execute(get_data)
            raw_datas_all = cursor.fetchall()
            print(f'{tenderType} 一共抓出 {len(raw_datas_all)} 筆還沒有內頁資料')
            if spilt:
                if spilt[1]>1:
                    first_part = int(len(raw_datas_all)/spilt[0]*(spilt[1]-1))
                    second_part = int(len(raw_datas_all)/spilt[0]*spilt[1])
                    raw_datas_all = raw_datas_all[first_part:second_part]
                    wording = f'{tenderType} 切分 {spilt[0]} 段，此為第 {spilt[1]} 段，取第 {first_part} 個到 {second_part} 個，共 {len(raw_datas_all)} 個，進行爬取'
                else:
                    wording = f'{tenderType} 不切分，共 {len(raw_datas_all)} 個，進行爬取'
            elif len(raw_datas_all)>0:
                wording = f'{tenderType} 不切分，共 {len(raw_datas_all)} 個，進行爬取'
            else:
                wording = f'{tenderType} 資料皆已爬取完成'
            raw_datas=raw_datas_all[0:block_crawl_no]
            print(wording)
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", str(ip) + " start", tenderType, wording))

        try:
            #限制嘗試抓取次數，避免死機
            x=0
            while len(raw_datas)>0:
                ll = get_only_detail(tenderType,raw_datas,diff_seconds_data,diff_seconds_conn,headers, db_settings,path,No_proxy=False)
                # get_only_detail會回傳成功抓取的筆數 (ll)
                if ll>=1:
                    x = 0
                else:
                    x = x + 1 
                    
                #避免過長抓取失敗
                del raw_datas_all[0:block_crawl_no]
                raw_datas=raw_datas_all[0:block_crawl_no]
                    
#                 conn = pymysql.connect(**db_settings)
#                 # 建立Cursor物件
#                 with conn.cursor(pymysql.cursors.DictCursor) as cursor:
#                   #資料表相關操作
#                     cursor.execute(get_data)
#                     raw_datas = cursor.fetchall()
#                 if len(raw_datas) ==0:
#                     print('都抓完了！！！！')
#                     break
#                 else:
                if x>=2:
                    print(f'其他錯誤_8：{tenderType} 出現 {len(raw_datas)} 筆無法抓取內頁資料')
                    break
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", str(ip)+"finish", tenderType, ""))

        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_detail", "wrong", "tenderDeclaration_final_detail", errMsg))
        
        if tenderType=='tenderDeclaration':
            del progress_tenderDeclaration_detail
        elif tenderType=='final':
            del progress_final_detail
            
            
    #限制取得內頁資料的時間 (若成功抓取則等待)
    diff_seconds_data = config_diff_seconds_data
    print(f"diff_seconds_data：{diff_seconds_data}")
    #限制每一次嘗試取得內頁資料的時間 (每次嘗試連接都等待)
    diff_seconds_conn = config_diff_seconds_conn
    print(f"diff_seconds_conn：{diff_seconds_conn}")
    #設定用本機ip爬取內頁資料
    No_proxy = config_No_proxy
    print(f"No_proxy：{No_proxy}")
    #爬取幾筆就存進資料庫？
    block_crawl_no = config_block_crawl_no
    print(f"block_crawl_no：{block_crawl_no}")
    spilt = False
    if request.args:
        dd = request.args.to_dict()
        print(dd)
        if set(['spilt']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許 spilt",500
        try:
            #切分段數
            spilt = dicMemberCheck('spilt',dd)
            if spilt !="" :
                spilt=eval(spilt)
                if len(spilt)==2 and spilt[0]>=spilt[1]:
                    print(f"spilt {spilt}")
            else:
                spilt = False
            print(f"spilt {spilt}")
        except Exception as ex:
            print(ex)
            return "參數轉換錯誤",500
    
    delete_ip(config_ip_time_notConnect,db_settings)
    if str(threading.enumerate()).find("thread_tenderDeclaration_detail") >= 0 :
        wording1="thread_tenderDeclaration_detail 執行緒正在執行"
    else:
        thread_tenderDeclaration_detail = threading.Thread(target=do_work_detail, kwargs={'tenderType':'tenderDeclaration','diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings,'No_proxy':No_proxy,'spilt':spilt,'block_crawl_no':block_crawl_no,'path':config_path},name="thread_tenderDeclaration_detail")
        thread_tenderDeclaration_detail.start()
        wording1="啟動 thread_tenderDeclaration_detail 執行緒"
        
    if str(threading.enumerate()).find("thread_final_detail") >= 0 :
        wording2="thread_final_detail 執行緒正在執行"
    else:
        thread_final_detail = threading.Thread(target=do_work_detail, kwargs={'tenderType':'final','diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings,'No_proxy':No_proxy,'spilt':spilt,'block_crawl_no':block_crawl_no,'path':config_path},name="thread_final_detail")
        thread_final_detail.start()
        wording2="啟動 thread_final_detail 執行緒"
        
    if str(threading.enumerate()).find("thread_searchAppeal_detail") >= 0 :
        wording2="thread_final_detail 執行緒正在執行"
    else:
        thread_final_detail = threading.Thread(target=do_work_detail, kwargs={'tenderType':'searchAppeal','diff_seconds_data':diff_seconds_data,'diff_seconds_conn':diff_seconds_conn,'headers':headers,'db_settings': db_settings,'No_proxy':No_proxy,'spilt':spilt,'block_crawl_no':block_crawl_no,'path':config_path},name="thread_searchAppeal_detail")
        thread_final_detail.start()
        wording2="啟動 thread_searchAppeal_detail 執行緒"
        
    return wording1 +" "+ wording2,200



@app.route('/progress', methods=['GET'])
def get_progress():
    wording={}
    try:
        wording['tenderDeclaration'] = progress_tenderDeclaration
    except:
        wording['tenderDeclaration'] = "tenderDeclaration 任務未啟動"
        
    try:
        wording['searchAppeal'] = progress_searchAppeal
    except:
        wording['searchAppeal'] = "searchAppeal 任務未啟動"
        
    try:
        wording['publicRead'] = progress_publicRead
    except:
        wording['publicRead'] = "publicRead 任務未啟動"
        
    try:
        wording['predict'] = progress_predict
    except:
        wording['predict'] = "predict 任務未啟動"
    
    try:
        wording['final'] = progress_final
    except:
        wording['final'] = "final 任務未啟動"
        
    try:
        wording['date_serach'] = progress_date_serach
    except:
        wording['date_serach'] = "date_serach 任務未啟動"

    try:
        wording['tenderDeclaration_detail'] = progress_tenderDeclaration_detail
    except:
        wording['tenderDeclaration_detail'] = "tenderDeclaration_detail 任務未啟動"
        
    try:
        wording['final_detail'] = progress_final_detail
    except:
        wording['final'] = "final_detail 任務未啟動"
        
    return wording,200

# @app.route('/test', methods=['GET'])
# def tesddt():
#     if request.args:
#         dd= request.args.to_dict()
#         if set(['start_date','end_date','']) >= set(dd.keys()):
#             return "傳參成功",200
#         else:
#             return "傳參失敗",200


@app.route('/date_search_crawler', methods=['GET'])
def date_serach():
    if request.args:
        dd= request.args.to_dict()
        if set(['start_date','end_date','if_publish']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' start_date '+'end_date '+'if_publish',500
    headers["User-Agent"] = UserAgent(use_cache_server=False).random
    def do_work_2(start_date,end_date,if_publish,db_settings,headers):
        s = rq.session()
        try:
            xx = search_by_date(s,start_date, end_date,if_publish,headers)
            print(xx)
            xxx = date_search_crawler(s,xx,headers)
            send_db_2(xxx,"date_search_by_date", db_settings)
        except Exception as ex:
            print(ex)
            error_class = ex.__class__.__name__ #取得錯誤類型
            detail = ex.args[0] #取得詳細內容
            cl, exc, tb = sys.exc_info() #取得Call Stack
            lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
            fileName = lastCallStack[0] #取得發生的檔案名稱
            lineNum = lastCallStack[1] #取得發生的行號
            funcName = lastCallStack[2] #取得發生的函數名稱
            errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
            print(errMsg)
            conn = pymysql.connect(**db_settings)
            with conn.cursor() as cursor:
                cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", "wrong", "date_search_crawler", error_class + detail + errMsg))
        conn = pymysql.connect(**db_settings)
        with conn.cursor() as cursor:
            cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " finish", "date_search_crawler", ""))
        
        
    try:
        if str(threading.enumerate()).find("thread_start_date_search_crawler") >=0:
            return "當前執行緒正在執行",500
    except Exception as ex:
        print(ex)
        pass
    
    try:
        start_date = str(request.args.get("start_date"))
        start_date = datetime.datetime.strptime(start_date, "%Y%m%d")
    except Exception as ex:    
        return "start_date 參數格式錯誤",500
    
    try:
        end_date = str(request.args.get("end_date"))
        end_date = datetime.datetime.strptime(end_date, "%Y%m%d")
    except Exception as ex:    
        return "end_date 參數格式錯誤",500
    
    try:
        if_publish = str(request.args.get("if_publish"))
        if not (if_publish == "yes" or if_publish == "no" or if_publish == "all"):
            return "if_publish 參數格式錯誤",500
    except Exception as ex:
        return "if_publish 參數沒帶",500
        
    
    ip = request.remote_addr
#     conn = pymysql.connect(**db_settings)
#     with conn.cursor() as cursor:
#         if_finish_command="SELECT status FROM log where task ='api_process_date_serach' order by UID DESC"
#         cursor.execute(if_finish_command)
#         if cursor.fetchone() is None:
#             cursor.execute(command_log, (datetime.datetime.now(), "api_process_date_serach", "initial", "", ""))
#         cursor.execute(if_finish_command)
#         if_finish = str(cursor.fetchone()[0])
    
#     print(if_finish)    
#     if re.search(r'finish', if_finish, flags=0) or re.search(r'wrong', if_finish, flags=0) or if_finish == "initial":
#         pass
#     else:
#         wording ="資料庫未收到結束log"
#         status_code = 500
#         return wording,status_code
    
    conn = pymysql.connect(**db_settings)
    with conn.cursor() as cursor:
        cursor.execute(command_log, (datetime.datetime.now(), "api_process_list", str(ip) + " start", "date_search_crawler", ""))
    
    thread_date_search_crawler = threading.Thread(target=do_work_2, kwargs={'start_date': start_date,'end_date': end_date,'if_publish':if_publish,'db_settings':db_settings,'headers':headers},name="thread_date_search_crawler")
    thread_date_search_crawler.start()    
    return "started",200


if __name__ == '__main__':
    app.debug = False
    app.config['JSON_AS_ASCII'] = False
    app.run(host=my_host, port=5000)
    #serve(app, host=my_host, port=5000)

my_host：127.0.0.1
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2022 14:33:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2022 14:33:16] "GET /start_task_detail HTTP/1.1" 200 -


diff_seconds_data：0.0
diff_seconds_conn：0.0
No_proxy：False
block_crawl_no：1000
ip_time_notConnect：25
searchAppeal 一共抓出 0 筆還沒有內頁資料
searchAppeal 資料皆已爬取完成
final 一共抓出 969 筆還沒有內頁資料
final 不切分，共 969 個，進行爬取
tenderDeclaration 一共抓出 1675 筆還沒有內頁資料
tenderDeclaration 不切分，共 1675 個，進行爬取
ip_qual_ratio：0.3
ip_time_use_interval：800.0
ip_qual_ratio：0.3
ip_time_use_interval：800.0
--------------------
抽取第 4 個，目前共有 503 個 proxy--------------------

抽取第 345 個，目前共有 503 個 proxy
http://102.68.134.94:8080 連接失敗，還剩餘 502 個proxysocks4://185.70.218.13:42707 連接失敗，還剩餘 502 個proxy
抽取第 100 個，目前共有 502 個 proxy

抽取第 428 個，目前共有 502 個 proxy
http://165.225.35.55:10605 連接失敗，還剩餘 501 個proxy
抽取第 276 個，目前共有 501 個 proxy
socks4://121.66.198.76:4145 連接失敗，還剩餘 500 個proxy
抽取第 397 個，目前共有 500 個 proxy
socks4://38.142.63.146:31596 連接失敗，還剩餘 501 個proxy
抽取第 312 個，目前共有 501 個 proxy
socks4://216.215.126.106:48324 連接失敗，還剩餘 499 個proxy
抽取第 430 個，目前共有 499 個 proxy
socks4://169.255.139.5:60279 連接失敗，還剩餘 500 個proxy
抽取第 202 個，目前共有 500 個 proxy
socks4://43.128.

3.15.83.21
progress_tenderDeclaration_detail 目前是第 19 項，抓取該細項內頁花費 0:00:08.852310 秒，進度：1.9%，總共有 1000 項
--------------------
抽取第 64 個，目前共有 478 個 proxy
socks4://36.66.16.23:5678 連接失敗，還剩餘 472 個proxy
抽取第 215 個，目前共有 472 個 proxy
3.15.83.21
progress_tenderDeclaration_detail 目前是第 20 項，抓取該細項內頁花費 0:00:08.628687 秒，進度：2.0%，總共有 1000 項
--------------------
抽取第 64 個，目前共有 478 個 proxy
3.15.83.4
progress_tenderDeclaration_detail 目前是第 21 項，抓取該細項內頁花費 0:00:07.887595 秒，進度：2.1%，總共有 1000 項
--------------------
抽取第 64 個，目前共有 478 個 proxy
socks4://103.166.38.65:3629 連接失敗，還剩餘 471 個proxy
抽取第 342 個，目前共有 471 個 proxy
3.15.25
progress_tenderDeclaration_detail 目前是第 22 項，抓取該細項內頁花費 0:00:08.986331 秒，進度：2.2%，總共有 1000 項
--------------------
抽取第 64 個，目前共有 478 個 proxy
socks4://190.221.152.130:56142 連接失敗，還剩餘 470 個proxy
抽取第 347 個，目前共有 470 個 proxy
3.15.25.1
progress_tenderDeclaration_detail 目前是第 23 項，抓取該細項內頁花費 0:00:09.854064 秒，進度：2.3%，總共有 1000 項
--------------------
抽取第 64 個，目前共有 478 個 proxy
3.15.25.1
progress_tenderDeclaration_de

http://165.225.34.64:10605 連接失敗，還剩餘 430 個proxy
抽取第 183 個，目前共有 430 個 proxy
socks4://31.146.161.194:5678 連接失敗，還剩餘 458 個proxy
抽取第 426 個，目前共有 458 個 proxy
socks4://80.191.169.76:4145 連接失敗，還剩餘 457 個proxy
抽取第 71 個，目前共有 457 個 proxy
http://165.225.208.105:10605 連接失敗，還剩餘 456 個proxy
抽取第 146 個，目前共有 456 個 proxy
socks4://101.109.251.42:4145 連接失敗，還剩餘 429 個proxy
抽取第 2 個，目前共有 429 個 proxy
http://191.5.163.149:9812 連接失敗，還剩餘 455 個proxy
抽取第 72 個，目前共有 455 個 proxy
http://165.225.208.116:10605 連接失敗，還剩餘 454 個proxy
抽取第 316 個，目前共有 454 個 proxy
http://102.68.128.217:8080 連接失敗，還剩餘 428 個proxy
抽取第 59 個，目前共有 428 個 proxy
socks4://187.44.146.2:32139 連接失敗，還剩餘 453 個proxy
抽取第 42 個，目前共有 453 個 proxy
http://157.100.53.99:999 連接失敗，還剩餘 427 個proxy
抽取第 306 個，目前共有 427 個 proxy
http://122.116.20.91:8080 連接失敗，還剩餘 452 個proxy
抽取第 111 個，目前共有 452 個 proxy
http://165.225.56.75:10605 連接失敗，還剩餘 451 個proxy
抽取第 18 個，目前共有 451 個 proxy
socks4://187.60.66.45:5678 連接失敗，還剩餘 426 個proxy
抽取第 424 個，目前共有 426 個 proxy
http://104.129.204.105:10605 連接失敗，還剩餘 4

...

決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDIwOTY= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDIwOTY= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="display:none">
		    
		</tr>
5.4.20
progress_final_detail 目前是第 722 項，抓取該細項內頁花費 0:00:02.940594 秒，進度：74.50980392156863%，總共有 969 項
--------------------
抽取第 1 個，目前共有 6 個 proxy
3.76.56.29.1
progress_tenderDeclaration_detail 目前是第 922 項，抓取該細項內頁花費 0:00:20.381647 秒，進度：92.2%，總共有 1000 項
--------------------
抽取第 8 個，目前共有 9 個 proxy
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDI0MDc= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https:

決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDIzOTA= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDIzOTA= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="display:none">
		    
		</tr>
3.76.55.60
progress_final_detail 目前是第 735 項，抓取該細項內頁花費 0:00:03.107079 秒，進度：75.85139318885449%，總共有 969 項
--------------------
抽取第 1 個，目前共有 6 個 proxy
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDEzMTI= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDEzMTI= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="d

決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDE3MjY= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDE3MjY= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="display:none">
		    
		</tr>
3.76.45.97.15
progress_final_detail 目前是第 750 項，抓取該細項內頁花費 0:00:03.197906 秒，進度：77.39938080495357%，總共有 969 項
--------------------
抽取第 1 個，目前共有 6 個 proxy
http://93.185.123.154:3128 連接失敗，還剩餘 8 個proxy
抽取第 1 個，目前共有 8 個 proxy
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDE2OTk= ，第4個 table，第13個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
3.76.45.97.15
progress_final_detail 目前是第 751 項，抓取該細項內頁花費 0:00:03.385262 秒，進度：77.50257997936016%，總共有

決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDM2MjM= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDM2MjM= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="display:none">
		    
		</tr>
3.76.45
progress_final_detail 目前是第 760 項，抓取該細項內頁花費 0:00:27.470924 秒，進度：78.43137254901961%，總共有 969 項
--------------------
抽取第 0 個，目前共有 1 個 proxy
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDM2MDU= ，第4個 table，第14個 tr
<tr>
													<td colspan="2" style="border:0px">
														<hr/>
													</td>
												</tr>
決標公告 內頁資料 解析失敗，出現在 https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDM2MDU= ，第5個 table，第8個 tr
<tr class="tb_s06" id="overEst" style="disp

3.87.8.91
progress_tenderDeclaration_detail 目前是第 960 項，抓取該細項內頁花費 0:01:43.391522 秒，進度：96.0%，總共有 1000 項
--------------------
抽取第 4 個，目前共有 5 個 proxy
http://93.185.123.154:3128 連接失敗，還剩餘 8 個proxy
抽取第 2 個，目前共有 8 個 proxy
3.87.70
progress_tenderDeclaration_detail 目前是第 961 項，抓取該細項內頁花費 0:00:20.906928 秒，進度：96.1%，總共有 1000 項
--------------------
抽取第 4 個，目前共有 5 個 proxy
http://128.199.214.87:3128 連接失敗，還剩餘 7 個proxy
抽取第 2 個，目前共有 7 個 proxy
3.87.70
progress_tenderDeclaration_detail 目前是第 962 項，抓取該細項內頁花費 0:00:34.782676 秒，進度：96.2%，總共有 1000 項
--------------------
抽取第 4 個，目前共有 5 個 proxy
http://151.106.13.221:1080 連接失敗，還剩餘 6 個proxy
抽取第 1 個，目前共有 6 個 proxy
http://121.156.109.108:8080 連接失敗，還剩餘 5 個proxy
抽取第 2 個，目前共有 5 個 proxy
http://157.100.12.138:999 內頁資料錯誤，還剩餘 4 個proxy
https://web.pcc.gov.tw/tps/atm/AtmAwardWithoutSso/QueryAtmAwardDetail?pkAtmMain=NTM3NDE5MzI=
抽取第 1 個，目前共有 4 個 proxy
http://154.49.216.33:3128 連接失敗，還剩餘 3 個proxy
抽取第 1 個，目前共有 3 個 proxy
3.87.9.8
progress_tenderDeclaration_detail 目前是第 963 項，抓取該細項內頁花費 